In [1]:
relative_path = r"tare_model_functions_v1_4_1.ipynb"
%run $relative_path

In [14]:
import os

In [15]:
project_root = os.path.abspath(os.getcwd())

In [21]:
filename = 'aeo_projections_2022_2050.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_hdd_projection_factors = pd.read_excel(io=file_path, sheet_name='hdd_factors_2018_2050')
df_fuelPrices_projection_factors = pd.read_excel(io=file_path, sheet_name='fuel_price_factors_2022_2050')

In [18]:
fuelOil_factors = calculate_fossilFuel_emission_factor("fuelOil", 0.0015, 0.1300, 0.83, 161.0, 1000, 138500)
naturalGas_factors = calculate_fossilFuel_emission_factor("naturalGas", 0.0006, 0.0922, 1.9, 117.6, 1000000, 1039)
propane_factors = calculate_fossilFuel_emission_factor("propane", 0.0002, 0.1421, 0.17, 136.6, 1000, 91452)

all_factors = {**fuelOil_factors, **naturalGas_factors, **propane_factors}

df_margEmis_factors = pd.DataFrame.from_dict(all_factors, orient="index", columns=["value"])
df_margEmis_factors.reset_index(inplace=True)
df_margEmis_factors.columns = ["pollutant", "value"]
df_margEmis_factors[["fuel_type", "pollutant"]] = df_margEmis_factors["pollutant"].str.split("_", expand=True)
df_margEmis_factors["unit"] = "[lb/kWh]"

# Add the 'state' column and assign 'National' to every row
df_margEmis_factors = df_margEmis_factors.assign(state='National')

df_margEmis_factors = df_margEmis_factors[["state", "fuel_type", "pollutant", "value", "unit"]]
df_margEmis_factors

,state,fuel_type,pollutant,value,unit
0,National,fuelOil,so2,5.118000e-06,[lb/kWh]
1,National,fuelOil,nox,4.435600e-04,[lb/kWh]
2,National,fuelOil,pm25,2.044736e-05,[lb/kWh]
3,National,fuelOil,co2,5.493320e-01,[lb/kWh]
4,National,naturalGas,so2,2.047200e-06,[lb/kWh]
5,National,naturalGas,nox,3.145864e-04,[lb/kWh]
6,National,naturalGas,pm25,6.239461e-06,[lb/kWh]
7,National,naturalGas,co2,4.012512e-01,[lb/kWh]
8,National,propane,so2,6.824000e-07,[lb/kWh]
9,National,propane,nox,4.848452e-04,[lb/kWh]


In [20]:
filename = 'easiur_National2024-06-1420-17.csv'
relative_path = os.path.join(r"margDamages_EASIUR/easiur_batchConversion_download", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margSocialCosts = pd.read_csv(file_path)
df_margSocialCosts

Retrieved data for filename: easiur_National2024-06-1420-17.csv
Located at filepath: /mnt/c/Users/Arnav/Documents/Research/Trane_Technologies/cmu-tare-model/margDamages_EASIUR/easiur_batchConversion_download/easiur_National2024-06-1420-17.csv




FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/Arnav/Documents/Research/Trane_Technologies/cmu-tare-model/margDamages_EASIUR/easiur_batchConversion_download/easiur_National2024-06-1420-17.csv'

In [22]:
filename = 'fuel_prices_nominal.csv'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_perkWh = pd.read_csv(file_path)
factor_dict = df_fuelPrices_projection_factors.set_index(['region', 'fuel_type', 'scenario']).to_dict('index')
iraRef_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'AEO2023 Reference case'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_iraRef = pd.concat([df_fuelPrices_perkWh, iraRef_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the scenario included
iraRef_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_iraRef, 'AEO2023 Reference case')
iraHigh_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'High Uptake of Inflation Reduction Act'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_iraHigh = pd.concat([df_fuelPrices_perkWh, iraHigh_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the scenario included
iraHigh_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_iraHigh, 'High Uptake of Inflation Reduction Act')


KeyError: 'census_division'

In [23]:
filename = "rsMeans_cityCostIndex.csv"
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_rsMeans_cityCostIndex = pd.read_csv(file_path)

df_rsMeans_cityCostIndex = pd.DataFrame({
    'State': df_rsMeans_cityCostIndex['State'],
    'City': df_rsMeans_cityCostIndex['City'],
    'Material': (df_rsMeans_cityCostIndex['Material']).round(2),
    'Installation': (df_rsMeans_cityCostIndex['Installation']).round(2),
    'Average': (df_rsMeans_cityCostIndex['Average']).round(2),
})
df_rsMeans_cityCostIndex

Retrieved data for filename: rsMeans_cityCostIndex.csv
Located at filepath: /mnt/c/Users/Arnav/Documents/Research/Trane_Technologies/cmu-tare-model/inflation_data/rsMeans_cityCostIndex.csv




,State,City,Material,Installation,Average
0,United States,+30 City Average,4.20,1.20,3.00
1,Alabama,Birmingham,5.18,0.42,3.38
2,Alabama,Huntsville,4.97,0.56,3.38
3,Alabama,Mobile,5.41,0.29,3.55
4,Alabama,Montgomery,5.44,0.98,3.87
...,...,...,...,...,...
286,Wisconsin,Milwaukee,4.02,4.15,4.11
287,Wisconsin,Racine,4.96,0.00,2.76
288,Wyoming,Casper,6.04,0.00,4.05
289,Wyoming,Cheyenne,6.02,0.14,3.87


In [24]:
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()
bls_cpi_annual_2023 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2023)].item()

# Precompute constant values
cpi_ratio_2023_2022 = bls_cpi_annual_2023 / bls_cpi_annual_2022
cpi_ratio_2023_2021 = bls_cpi_annual_2023 / bls_cpi_annual_2021  # This will be 1
cpi_ratio_2023_2020 = bls_cpi_annual_2023 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2023_2019 = bls_cpi_annual_2023 / bls_cpi_annual_2019 
cpi_ratio_2023_2018 = bls_cpi_annual_2023 / bls_cpi_annual_2018 
cpi_ratio_2023_2013 = bls_cpi_annual_2023 / bls_cpi_annual_2013
cpi_ratio_2023_2010 = bls_cpi_annual_2023 / bls_cpi_annual_2010
cpi_ratio_2023_2008 = bls_cpi_annual_2023 / bls_cpi_annual_2008  # For EPA VSL and SCC

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: /mnt/c/Users/Arnav/Documents/Research/Trane_Technologies/cmu-tare-model/inflation_data/bls_cpiu_2005-2023.xlsx


In [3]:
df_enduse_compare()

TypeError: df_enduse_compare() missing 4 required positional arguments: 'df_mp', 'input_mp', 'menu_mp', and 'df_baseline'

In [2]:
project_future_consumption()

TypeError: project_future_consumption() missing 3 required positional arguments: 'df', 'hdd_factor_lookup', and 'menu_mp'

In [4]:
calculate_marginal_damages()

TypeError: calculate_marginal_damages() missing 1 required positional argument: 'df'

In [5]:
calculate_public_npv()

TypeError: calculate_public_npv() missing 2 required positional arguments: 'df' and 'menu_mp'

In [6]:
calculate_annual_fuelCost()

TypeError: calculate_annual_fuelCost() missing 4 required positional arguments: 'df', 'fuel_price_lookup', 'menu_mp', and 'scenario'

In [7]:
map_average_cost()

TypeError: map_average_cost() missing 1 required positional argument: 'city'

In [8]:
obtain_heating_system_specs()

TypeError: obtain_heating_system_specs() missing 1 required positional argument: 'df'

In [9]:
calculate_installation_cost()

TypeError: calculate_installation_cost() missing 5 required positional arguments: 'df', 'cost_dict', 'rsMeans_national_avg', 'menu_mp', and 'end_use'

In [10]:
calculate_replacement_cost()

TypeError: calculate_replacement_cost() missing 5 required positional arguments: 'df', 'cost_dict', 'rsMeans_national_avg', 'menu_mp', and 'end_use'

In [11]:
calculate_heating_installation_premium()

TypeError: calculate_heating_installation_premium() missing 3 required positional arguments: 'df', 'rsMeans_national_avg', and 'cpi_ratio_2023_2013'

In [12]:
calculate_private_NPV()

TypeError: calculate_private_NPV() missing 4 required positional arguments: 'df', 'interest_rate', 'input_mp', and 'menu_mp'

In [13]:
adoption_decision()

TypeError: adoption_decision() missing 1 required positional argument: 'df'